In [2]:
import pandas as pd
import numpy as np
import os

In [3]:
path2021 = 'D:\\cityDNA\\beijing_20210101-20211231'
path2022 = 'D:\\cityDNA\\beijing_20220101-20221231'

In [129]:
## list all target files with string EXTRA
### 2021/2022均可用，修改参数'path202X'为上一步中的变量即可
files = os.listdir(path202X)
tgtfiles = []
for file in files:
    if file[8] == 'e':
        tgtfiles.append(os.path.join(path202X,file))
# tgtfiles


In [120]:
## test module,非必要不需要用，可跳过
# tst = pd.read_csv(tgtfiles[0])
# tst.columns
# tst = tst[['date','type','官园','房山']]

# # 将'date'列转换为datetime类型，以便更灵活地进行分组
# tst['date'] = pd.to_datetime(tst['date'], format='%Y%m%d')

# # 计算每天每项指标的均值
# daily_means = tst.groupby(['date', 'type']).agg({'官园': 'mean', '房山': 'mean'}).reset_index()
# final = daily_means.drop(index=[1,3,5,7]).reset_index(drop=True)
# final

Index(['date', 'hour', 'type', '东城东四', '东城天坛', '西城官园', '西城万寿西宫', '朝阳奥体中心',
       '朝阳农展馆', '海淀万柳', '海淀四季青', '丰台小屯', '丰台云岗', '石景山古城', '石景山老山', '昌平镇',
       '昌平南邵', '定陵(对照点)', '延庆夏都', '延庆石河营', '怀柔镇', '怀柔新城', '密云镇', '密云新城', '平谷镇',
       '平谷新城', '顺义新城', '顺义北小营', '通州永顺', '通州东关', '大兴黄村', '大兴旧宫', '亦庄开发区',
       '京东南区域点', '门头沟双峪', '门头沟三家店', '房山良乡', '房山燕山'],
      dtype='object')

In [125]:
### 2021公式
def data_mean2021(num):
    # important!!!发现前18天数据中只有"官园","房山"数据,后续数据记录均为"西城官园"，"房山良乡"数据
    if num < 18:
        daily_file = pd.read_csv(tgtfiles[num])
        daily_file = daily_file[['date','type','官园','房山']]

        # 将'date'列转换为datetime类型，以便更灵活地进行分组
        daily_file['date'] = pd.to_datetime(daily_file['date'], format='%Y%m%d')

        # 计算每天每项指标的均值
        daily_means = daily_file.groupby(['date', 'type']).agg({'官园': 'mean', '房山': 'mean'}).reset_index()
        daily_means_clean = daily_means.drop(index=[1,3,5,7]).reset_index(drop=True)
    else:
        daily_file = pd.read_csv(tgtfiles[num])
        daily_file = daily_file[['date','type','西城官园','房山良乡']]

        # 将'date'列转换为datetime类型，以便更灵活地进行分组
        daily_file['date'] = pd.to_datetime(daily_file['date'], format='%Y%m%d')

        # 计算每天每项指标的均值
        daily_means = daily_file.groupby(['date', 'type']).agg({'西城官园': 'mean', '房山良乡': 'mean'}).reset_index()
        daily_means_clean = daily_means.drop(index=[1,3,5,7]).reset_index(drop=True)
        daily_means_clean = daily_means_clean.rename(columns={'西城官园': '官园','房山良乡':'房山'})
    return daily_means_clean

In [123]:
### 2022公式
def data_mean2022(num):
    daily_file = pd.read_csv(tgtfiles[num])
    daily_file = daily_file[['date','type','西城官园','房山良乡']]

    # 将'date'列转换为datetime类型，以便更灵活地进行分组
    daily_file['date'] = pd.to_datetime(daily_file['date'], format='%Y%m%d')

    # 计算每天每项指标的均值
    daily_means = daily_file.groupby(['date', 'type']).agg({'西城官园': 'mean', '房山良乡': 'mean'}).reset_index()
    daily_means_clean = daily_means.drop(index=[1,3,5,7]).reset_index(drop=True)
    daily_means_clean = daily_means_clean.rename(columns={'西城官园': '官园','房山良乡':'房山'})
    return daily_means_clean

In [130]:
# 创建一个空列表df, 用于拼接调用公式提取出来的暂存表tmp_df
df = pd.DataFrame({'date':[],'type':[],'官园':[],'房山':[]})

for i in range(len(tgtfiles)):
    ### 此处根据不同年份调用不同function
    tmp_df = data_mean2022(i)
    df = pd.concat([df,tmp_df])
# df


C:\Users\Nevin JI\AppData\Local\Temp\ipykernel_18600\3627730298.py:6: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df,tmp_df])


,date,type,官园,房山
0,2022-01-01,CO,0.575000,0.470833
1,2022-01-01,NO2,42.708333,39.625000
2,2022-01-01,O3,27.458333,28.391304
3,2022-01-01,SO2,6.750000,4.041667
0,2022-01-02,CO,0.254167,0.262500
...,...,...,...,...
3,2022-12-30,SO2,4.409091,4.166667
0,2022-12-31,CO,0.633333,0.654167
1,2022-12-31,NO2,49.666667,45.391304
2,2022-12-31,O3,15.476190,14.956522


In [131]:
## 将上一步df输出为csv文件，记得修改路径名称
df.to_csv(os.path.join(path2022,'2022anniversary.csv'))